# KHOA LUAN VIETSTOCK
Huynh Duc Huy

In [1]:
from __future__ import print_function
import time
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, merge, BatchNormalization,GRU
from keras.datasets import imdb

import os
from keras.preprocessing.text import Tokenizer

Using Theano backend.


In [2]:
max_features =20000
max_len = 200  # cut texts after this number of words (among top max_features most common words)

In [3]:
# get dataset and unzip: http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

X_train = []
y_train = []

path = 'C:/2_MyThesis/ThucNghiem/dataset/mau3/out/train/pos/'
X_train.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_train.extend([1 for _ in range(776)])

path = 'C:/2_MyThesis/ThucNghiem/dataset/mau3/out/train/neg/'
X_train.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_train.extend([0 for _ in range(543)])


In [4]:
# read in the test data

X_test = []
y_test = []

path = 'C:/2_MyThesis/ThucNghiem/dataset/mau3/out/test/pos/'
X_test.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_test.extend([1 for _ in range(317)])

path = 'C:/2_MyThesis/ThucNghiem/dataset/mau3/out/test/neg/'
X_test.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_test.extend([0 for _ in range(248)])


In [5]:
#tokenize works to list of integers where each integer is a key to a word
imdbTokenizer = Tokenizer(nb_words=max_features)

imdbTokenizer.fit_on_texts(X_train)

In [6]:
#print top 20 words 
#note zero is reserved for non frequent words
for word, value in imdbTokenizer.word_index.items():
    if value < 20:
        print(value, word)

2 trong
15 giá
12 cho
8 có
1 và
18 đã
14 đầu
4 đồng
13 công
7 các
17 được
3 của
11 tăng
5 năm
9 tỷ
16 doanh
19 cổ
6 với
10 là


In [7]:
#create int to word dictionary
intToWord = {}
for word, value in imdbTokenizer.word_index.items():
    intToWord[value] = word

#add a symbol for null placeholder
intToWord[0] = "!!!NA!!!"
    
#print(intToWord[1])
#print(intToWord[2])
#print(intToWord[32])

In [8]:
#convert word strings to integer sequence lists
#print(X_train[0])
#print(imdbTokenizer.texts_to_sequences(X_train[:1]))
#for value in imdbTokenizer.texts_to_sequences(X_train[:1])[0]:
    #print(intToWord[value])
    
X_train = imdbTokenizer.texts_to_sequences(X_train)
X_test = imdbTokenizer.texts_to_sequences(X_test)

In [9]:
# Censor the data by having a max review length (in number of words)

#use this function to load data from keras pickle instead of munging as shown above
#(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features,
#                                                      test_split=0.2)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

1319 train sequences
565 test sequences
Pad sequences (samples x time)
X_train shape: (1319L, 200L)
X_test shape: (565L, 200L)


In [10]:
#example of a sentence sequence, note that lower integers are words that occur more commonly
#print("x:", X_train[0]) #per observation vector of 20000 words
#print("y:", y_train[0]) #positive or negative review encoding

In [11]:
# double check that word sequences behave/final dimensions are as expected
#print("y distribution:", np.unique(y_train, return_counts=True))
#print("max x word:", np.max(X_train), "; min x word", np.min(X_train))
#print("y distribution test:", np.unique(y_test, return_counts=True))
#print("max x word test:", np.max(X_test), "; min x word", np.min(X_test))

In [12]:
#print("most and least popular words: ")
#print(np.unique(X_train, return_counts=True))
# as expected zero is the highly used word for words not in index

In [13]:
#set model hyper parameters
epochs = 6
embedding_neurons = 100
lstm_neurons = 100
batch_size =32

In [14]:
# Forward Pass LSTM Network

# this is the placeholder tensor for the input sequences
sequence = Input(shape=(max_len,), dtype='int32')
# this embedding layer will transform the sequences of integers
# into vectors of size embedding
# embedding layer converts dense int input to one-hot in real time to save memory
embedded = Embedding(max_features, embedding_neurons, input_length=max_len)(sequence)
# normalize embeddings by input/word in sentence
bnorm = BatchNormalization()(embedded)

# apply forwards LSTM layer size lstm_neurons
forwards = GRU(lstm_neurons, dropout_W=0.6, dropout_U=0.4)(bnorm)

# dropout 
after_dp = Dropout(0.3)(forwards)
output = Dense(1, activation='sigmoid')(after_dp)

model_fdir_atom = Model(input=sequence, output=output)
# review model structure
print(model_fdir_atom.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 200, 100)      2000000     input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNormal(None, 200, 100)      200         embedding_1[0][0]                
____________________________________________________________________________________________________
gru_1 (GRU)                      (None, 100)           60300       batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [15]:
# Forward pass LSTM network

# try using different optimizers and different optimizer configs
model_fdir_atom.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
start_time = time.time()

history_fdir_atom = model_fdir_atom.fit(X_train, y_train,
                    batch_size=batch_size,
                    nb_epoch=epochs,
                    validation_data=[X_test, y_test], 
                    verbose=2)

end_time = time.time()
average_time_per_epoch = (end_time - start_time) / epochs
print("avg sec per epoch:", average_time_per_epoch)

Train...
Train on 1319 samples, validate on 565 samples
Epoch 1/6
18s - loss: 0.7411 - acc: 0.5299 - val_loss: 0.7092 - val_acc: 0.5398
Epoch 2/6
18s - loss: 0.6915 - acc: 0.5823 - val_loss: 0.7233 - val_acc: 0.5575
Epoch 3/6
19s - loss: 0.6464 - acc: 0.6391 - val_loss: 0.7383 - val_acc: 0.5611
Epoch 4/6
18s - loss: 0.6208 - acc: 0.6649 - val_loss: 0.7575 - val_acc: 0.5416
Epoch 5/6
18s - loss: 0.5857 - acc: 0.6892 - val_loss: 0.7739 - val_acc: 0.5381
Epoch 6/6
18s - loss: 0.5918 - acc: 0.6823 - val_loss: 0.8044 - val_acc: 0.5398
avg sec per epoch: 20.6148333152


In [16]:
# Bi-directional Atom

# based on keras tutorial: https://github.com/fchollet/keras/blob/master/examples/imdb_bidirectional_lstm.py

# this is the placeholder tensor for the input sequences
sequence = Input(shape=(max_len,), dtype='int32')
# this embedding layer will transform the sequences of integers
# into vectors of size embedding
# embedding layer converts dense int input to one-hot in real time to save memory
embedded = Embedding(max_features, embedding_neurons, input_length=max_len)(sequence)
# normalize embeddings by input/word in sentence
bnorm = BatchNormalization()(embedded)

# apply forwards LSTM layer size lstm_neurons
forwards = GRU(lstm_neurons, dropout_W=0.4, dropout_U=0.4)(bnorm)
# apply backwards LSTM
backwards = GRU(lstm_neurons, dropout_W=0.4, dropout_U=0.4, go_backwards=True)(bnorm)

# concatenate the outputs of the 2 LSTMs
merged = merge([forwards, backwards], mode='concat', concat_axis=-1)
after_dp = Dropout(0.5)(merged)
output = Dense(1, activation='sigmoid')(after_dp)

model_bidir_atom = Model(input=sequence, output=output)
# review model structure
print(model_bidir_atom.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 200, 100)      2000000     input_2[0][0]                    
____________________________________________________________________________________________________
batchnormalization_2 (BatchNormal(None, 200, 100)      200         embedding_2[0][0]                
____________________________________________________________________________________________________
gru_2 (GRU)                      (None, 100)           60300       batchnormalization_2[0][0]       
___________________________________________________________________________________________

In [17]:
# Bi-directional Atom

# try using different optimizers and different optimizer configs
model_bidir_atom.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
start_time = time.time()

history_bidir_atom = model_bidir_atom.fit(X_train, y_train,
                    batch_size=batch_size,
                    nb_epoch=epochs,
                    validation_data=[X_test, y_test], 
                    verbose=2)

end_time = time.time()
average_time_per_epoch = (end_time - start_time) / epochs
print("avg sec per epoch:", average_time_per_epoch)

Train...
Train on 1319 samples, validate on 565 samples
Epoch 1/6
31s - loss: 0.7607 - acc: 0.5550 - val_loss: 0.7134 - val_acc: 0.5398
Epoch 2/6
34s - loss: 0.6618 - acc: 0.6293 - val_loss: 0.7607 - val_acc: 0.5257
Epoch 3/6
32s - loss: 0.5829 - acc: 0.6907 - val_loss: 0.7818 - val_acc: 0.5434
Epoch 4/6
34s - loss: 0.4899 - acc: 0.7688 - val_loss: 0.8249 - val_acc: 0.5363
Epoch 5/6
35s - loss: 0.4251 - acc: 0.8127 - val_loss: 0.8714 - val_acc: 0.5398
Epoch 6/6
31s - loss: 0.3689 - acc: 0.8408 - val_loss: 0.9340 - val_acc: 0.5310
avg sec per epoch: 37.0578333537
